In this notebook we build pre-trained model to deploy in google cloud ai platform

## 1.Build and test model

In [ ]:
#import dependencies
import torch, torchvision

In [ ]:
print(f"torch = {torch.__version__}; cuda is avaliable:{torch.cuda.is_available()}")
print('==========')
!gcc --version
print('==========')
!nvcc --version

In [ ]:
# import some common libraries
import numpy as np
import os, json, cv2, random

In [ ]:
#install if needed
#!pip install pip install Pillow

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (50,100)
from PIL import Image

In [ ]:
#!git clone https://github.com/facebookresearch/detectron2.git
#!pip install -e detectron2

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
detectron2.__version__

In [ ]:
im = Image.open("test_photo.jpg")
plt.imshow(im);

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2 
cfg.MODEL.DEVICE = 'cpu' #cuda or cpu
cfg.MODEL.WEIGHTS = "model_final.pth"  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold

predictor = DefaultPredictor(cfg)

In [ ]:
#im = cv2.imread("test_photo.jpg")
im = cv2.imread("photos/6.jpg")
print(type(im))
im.shape

In [ ]:
%%time
#Делаем одиночное предсказание
outputs = predictor(im)  


In [ ]:
outputs

In [ ]:
%%time
v = Visualizer(im[:, :, ::-1], scale=1.)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

plt.imshow(out.get_image()[:, :, ::-1]);

### Convert to poligons

In [ ]:
import cv2

In [ ]:
pred_masks = outputs['instances'].pred_masks.numpy()
pred_masks = np.uint8(pred_masks)
pred_masks.shape

In [ ]:
def mask_to_rectangle(mask):
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    if len(contours) == 0:
        return []
    
    contour1 = contours[0]
    for i in range(1, len(contours)):
        contour2 = np.array(contours[i])
        contour1 = np.concatenate((contour1, contour2), axis=0)
    contour1 = contour1.squeeze()
    contour1 = cv2.convexHull(contour1)
    
    rect = cv2.boxPoints(cv2.minAreaRect(contour1))
    return rect

In [ ]:
%%time

boxes = []
for i in range(pred_masks.shape[0]):
    box = mask_to_rectangle(pred_masks[i])
    boxes.append(box)  
len(boxes)   

In [ ]:
boxes[0]

In [ ]:
def plot_blocks_on_photo(photo_path, boxes, save_photo=False):
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    import pylab
    from PIL import Image

    def build_polygon(points):
        xy = np.array(points)
        return patches.Polygon(xy, closed=True, linewidth=2, fill=True, alpha=0.5, color=np.random.rand(3, ))

    pylab.rcParams['figure.figsize'] = [50.0, 100.0]
    im = Image.open(photo_path)
    
    fig, ax = plt.subplots()
    ax.imshow(im)
    bx = fig.gca()

    for box in boxes:
        bx.add_patch(build_polygon(box))
    
    if save_photo:
        fig.savefig('predicted_photo.jpg')
    else:
        plt.show()

In [ ]:
plot_blocks_on_photo('photos/6.jpg',boxes)

## 2.Dump, load and test model

In [ ]:
import joblib

In [ ]:
joblib.dump(predictor, 'model/model.joblib')

In [ ]:
BUCKET_NAME="detectron-model"
REGION="us-central1"

!echo $REGION

In [ ]:
!gcloud config set project biblosphere-210106

In [ ]:
!gsutil cp ./model.joblib gs://$BUCKET_NAME/model.joblib

In [ ]:
predictor = joblib.load("./model.joblib")

In [ ]:
%time
#Делаем одиночное предсказание
file_name = 'test_photo.jpg'
im = cv2.imread(file_name)
outputs = predictor(im)  

In [ ]:
%time
v = Visualizer(im[:, :, ::-1], scale=1.)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

plt.imshow(out.get_image()[:, :, ::-1]);